In [2]:
import os, glob
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
import json
from shapely.geometry import Point

In [3]:
MAX_VAL_MAPBOX = 800
cmap = plt.get_cmap('turbo', 256)
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250212_hmineq1.0_tmin20050000_mean_grid_100.csv"
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250318_hmineq1.0_tmin20050000_mean_grid_100.csv"
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250407_hmineq1.0_tmin20050000_mean_grid_100.csv"
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250411_hmineq1.0_tmin20050000_mean_grid_100.csv"
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250425_hmineq1.0_tmin20050000_mean_grid_100.csv"
#ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250605_hmineq1.0_tmin20050000_mean_grid_100.csv"
ground_truth_data_file = f"/media/maffe/nvme/iceboost_train_dataset/iceboost_train_20250910_hmineq1.0_tmin20050000_mean_grid_100.csv"

dir_path = os.path.dirname(ground_truth_data_file)
file_name = os.path.basename(ground_truth_data_file)

ground_truth_df = pd.read_csv(f"{dir_path}/{file_name}")
ground_truth_df = ground_truth_df[['POINT_LAT', 'POINT_LON', 'THICKNESS']]

ground_truth_df['color'] = np.clip(ground_truth_df['THICKNESS'], 0, MAX_VAL_MAPBOX)
# Convert thickness to RGB colors (remove alpha channel)
colors = cmap(ground_truth_df['color'].values / MAX_VAL_MAPBOX)[:, :3] * 255
# Convert RGB to HEX
ground_truth_df['color'] = ['#{:02x}{:02x}{:02x}'.format(int(r), int(g), int(b)) for r, g, b in colors]

# Check result
print(ground_truth_df.head())
print(ground_truth_df['THICKNESS'].min(), ground_truth_df['THICKNESS'].max())

   POINT_LAT  POINT_LON  THICKNESS    color
0  46.958875   9.387544       10.5  #341b51
1  46.958842   9.387574        9.0  #32184a
2  46.958875   9.387574       10.5  #341b51
3  46.958842   9.387605        8.0  #32184a
4  46.958875   9.387605       11.0  #341b51
1.0 2739.624991861979


In [4]:
# Create GeoDataFrame with points efficiently
gdf = gpd.GeoDataFrame(
    ground_truth_df, 
    geometry=gpd.points_from_xy(ground_truth_df.POINT_LON, ground_truth_df.POINT_LAT),
    crs="EPSG:4326"
)

In [5]:
gdf.head()

,POINT_LAT,POINT_LON,THICKNESS,color,geometry
0,46.958875,9.387544,10.5,#341b51,POINT (9.38754 46.95888)
1,46.958842,9.387574,9.0,#32184a,POINT (9.38757 46.95884)
2,46.958875,9.387574,10.5,#341b51,POINT (9.38757 46.95888)
3,46.958842,9.387605,8.0,#32184a,POINT (9.38761 46.95884)
4,46.958875,9.387605,11.0,#341b51,POINT (9.38761 46.95888)


In [6]:
# Convert to GeoJSON format
geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": row["geometry"].__geo_interface__,  # Convert Shapely Point to GeoJSON
            "properties": {
                "thickness": row["THICKNESS"],
                "color": row["color"]
            }
        }
        for _, row in gdf.iterrows()
    ]
}

In [7]:
# Save to GeoJSON
geojson_file = file_name.replace(".csv", ".geojson")
with open(f"/home/maffe/PycharmProjects/iceboost_webapp/{geojson_file}", "w") as f:
    json.dump(geojson_data, f)

print(f"GeoJSON saved: {geojson_file}")

GeoJSON saved: iceboost_train_20250910_hmineq1.0_tmin20050000_mean_grid_100.geojson
